In [1]:
from pathlib import Path
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, explode
from pyspark.sql.types import StringType, IntegerType, BooleanType, FloatType, TimestampType

DATA_DIR = Path("data/ml-latest-small")

In [2]:
spark = SparkSession.builder\
    .master("local")\
    .appName("Word Count")\
    .getOrCreate()

In [3]:
# TODO: Spark schemas?
def read_df(file: str, types: dict):
    df = spark.read\
        .format("csv")\
        .option("header", "true")\
        .load(str(DATA_DIR / file))
    return set_dtypes(df, types)

def set_dtypes(df, types: dict):
    for column, dtype in types.items():
        dtype = {
            str: StringType(),
            int: IntegerType(),
            float: FloatType(),
            bool: BooleanType(),
            "time": TimestampType() 
        }[dtype]
        df = df.withColumn(column, col(column).cast(dtype))
    return df

df_links = read_df("links.csv", {
    "movieId": int,
    "imdbId": int,
    "tmdbId": int
})
df_movies = read_df("movies.csv", {
    "movieId": int,
    "title": str,
    "genres": str
})
df_ratings = read_df("ratings.csv", {
    "userId": int,
    "movieId": int,
    "rating": float,
    "timestamp": "time"
})
df_tags = read_df("tags.csv", {
    "userId": int,
    "movieId": int,
    "tag": str,
    "timestamp": "time"
})

In [7]:
# Given a user, get the number of movies watched per genre
def search_user(user_id: int):
    rated_movies = df_ratings.filter(df_ratings.userId == user_id).select("movieId").distinct()
    tagged_movies = df_tags.filter(df_tags.userId == user_id).select("movieId").distinct()
    movies = rated_movies.union(tagged_movies).distinct().join(df_movies, on=["movieId"], how="inner")
    movies = movies.select(movies.movieId, explode(split(movies.genres, "\|")).alias("genre"))
    movies = movies.groupBy("genre").count()
    return movies.collect()
search_user(1)

[Row(genre='Crime', count=45),
 Row(genre='Romance', count=26),
 Row(genre='Thriller', count=55),
 Row(genre='Adventure', count=85),
 Row(genre='Drama', count=68),
 Row(genre='War', count=22),
 Row(genre='Fantasy', count=47),
 Row(genre='Mystery', count=18),
 Row(genre='Musical', count=22),
 Row(genre='Animation', count=29),
 Row(genre='Film-Noir', count=1),
 Row(genre='Horror', count=17),
 Row(genre='Western', count=7),
 Row(genre='Comedy', count=83),
 Row(genre='Children', count=42),
 Row(genre='Action', count=90),
 Row(genre='Sci-Fi', count=40)]

In [24]:
df_ratings.filter("userId = 21").filter("movieId=1").collect()

[Row(userId=21, movieId=1, rating=3.5, timestamp=None)]

In [29]:
# Given a list of users, search all movies watched by each user
def search_movies_by_users(user_ids: [int]):
    rated_movies = df_ratings.filter(df_ratings.userId.isin(user_ids))
    tagged_movies = df_tags.filter(df_tags.userId.isin(user_ids))
    movies = rated_movies.join(tagged_movies, on=["userId", "movieId"], how="outer")
    movies = movies.select("userId", "movieId").distinct()
    movies = movies.groupBy("movieId").count().join(df_movies, on=["movieId"]).select("title", "count")
    return movies.collect()
search_movies_by_users([1, 21])

Row(title='Pride & Prejudice (2005)', count=1),
 Row(title='Who Framed Roger Rabbit? (1988)', count=1),
 Row(title='Pink Floyd: The Wall (1982)', count=1),
 Row(title='Logan (2017)', count=1),
 Row(title='Teenage Mutant Ninja Turtles III (1993)', count=1),
 Row(title='Ratatouille (2007)', count=1),
 Row(title='Transformers: The Movie (1986)', count=1),
 Row(title='Kingsman: The Secret Service (2015)', count=1),
 Row(title='The Interview (2014)', count=1),
 Row(title='Moonraker (1979)', count=1),
 Row(title='Monty Python and the Holy Grail (1975)', count=1),
 Row(title='Jurassic Park III (2001)', count=1),
 Row(title='Predator (1987)', count=1),
 Row(title='Dennis the Menace (1993)', count=1),
 Row(title='Collateral (2004)', count=1),
 Row(title='Welcome to Woop-Woop (1997)', count=1),
 Row(title='Big Lebowski, The (1998)', count=1),
 Row(title='Frozen (2013)', count=1),
 Row(title='Die Hard: With a Vengeance (1995)', count=1),
 Row(title='Terminator, The (1984)', count=2),
 Row(title='

In [35]:
# Search movie by id/title, show the average rating, the number of users that have watched the movie
def search_movies_watched_by_id(movie_id: int):
    rated_movies = df_ratings.where(f"movieId = {movie_id}")
    tagged_movies = df_tags.where(f"movieId = {movie_id}")
    movies = rated_movies.join(tagged_movies, on=["userId"], how="outer")
    movies = movies.select("userId").distinct()
    return movies.count()
search_movies_watched_by_id(1) # Toy Story

215

In [54]:
# Search movie by id/title, show the average rating, the number of users that have watched the movie
def search_movies_avg_rating_by_id(movie_id: int):
    rated_movies = df_ratings.where(f"movieId = {movie_id}").agg({"rating": "avg"})
    return rated_movies.first()[0]
search_movies_avg_rating_by_id(1) # Toy Story

3.9209302325581397

In [62]:
# Given a list of genres, search all movies belonging to each genre
def search_movies_by_genre(genres: [str]):
    movies = df_movies.select("movieId", "title", explode(split(df_movies.genres, "\|")).alias("genre"))
    movies = movies.filter(movies.genre.isin(genres)).dropDuplicates("movieId") # TODO: fix error with dropDuplicates
    return movies.collect()
search_movies_by_genre(["Action"])

pon a Time in China III (Wong Fei-hung tsi sam: Siwong tsangba) (1993)', genre='Action'),
 Row(movieId=6754, title='Underworld (2003)', genre='Action'),
 Row(movieId=6764, title='Rundown, The (2003)', genre='Action'),
 Row(movieId=6800, title='Cobra (1986)', genre='Action'),
 Row(movieId=6808, title='Where Eagles Dare (1968)', genre='Action'),
 Row(movieId=6812, title='Rookie, The (1990)', genre='Action'),
 Row(movieId=6814, title='City Heat (1984)', genre='Action'),
 Row(movieId=6835, title='Alien Contamination (1980)', genre='Action'),
 Row(movieId=6857, title='Ninja Scroll (Jûbei ninpûchô) (1995)', genre='Action'),
 Row(movieId=6872, title='House of the Dead, The (2003)', genre='Action'),
 Row(movieId=6874, title='Kill Bill: Vol. 1 (2003)', genre='Action'),
 Row(movieId=6882, title='Returner (Ritaanaa) (2002)', genre='Action'),
 Row(movieId=6893, title='Italian Job, The (1969)', genre='Action'),
 Row(movieId=6934, title='Matrix Revolutions, The (2003)', genre='Action'),
 Row(movieId